This SQL code was written utilizing Jupyter. This data was for a guided project in a certificate I am currently working on. Jupyter is a python coding program, so this project first needed to download the SQL coding tool. Each section starts with the command %% sql which tells Jupyter to run the following code as though it were SQL, not python. The first command box is where I load the SQL tool, and my dataset.

In [11]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [12]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


These are the first 5 rows from the CIA's "factbook" data set.

In [13]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Next the project wanted to know the spread of the population, as well as the population growth rate.

In [14]:
%%sql
SELECT MIN(population) min_pop, MAX(population) max_pop, MIN(population_growth) min_pop_gro, MAX(population_growth) max_pop_gro
  FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_gro,max_pop_gro
0,7256490011,0.0,4.02


Identifying the limits of the population variable, it seemed curious that a country would have a population of 0, and that another would have a populatino over 7 billion. The next line of code was used to identify what was happening here.

In [15]:
%%sql
SELECT name
  FROM facts
 WHERE (population = 0)
    OR (population = 7256490011);

 * sqlite:///factbook.db
Done.


name
Antarctica
World


We can now see that only one country has a population of 0, and that country is Antarctica. We can also see that our data has a country named "World." This is obviously not a country, and will be removed from all future calculations. I then reran the spread of population and population growth, removing the outlier, "World," from the calculations.

In [16]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE (name <> 'World');

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


To help identify countries of interest, we first calculated the averages for population and the size (area) of the countries in our data set. These will be used later as qualifying parameters.

In [17]:
%%sql
SELECT ROUND(AVG(population), 2), ROUND(AVG(area), 2)
  FROM facts
 WHERE (name <> 'World');

 * sqlite:///factbook.db
Done.


"ROUND(AVG(population), 2)","ROUND(AVG(area), 2)"
32242666.57,555093.55


To close out the guided project we wanted to identify countries which have populations greater than the average population, and were also smaller than the average country. This is where the guided project ended. I ordered these countries by their population in descending order.

In [19]:
%%sql
SELECT f.*
  FROM facts f
 WHERE (f.population > (SELECT AVG(population)
                         FROM facts
                        WHERE (name <> 'World')))
   AND (f.area < (SELECT AVG(area)
                    FROM facts
                   WHERE (name <> 'World')))
 ORDER BY population DESC;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Just out of curiosity I wanted to identify countries which have a high population density, an above average population, and a low population growth rate. To make this code simpler to follow I utilized CTE's to write it. I started by identifying the 30 countries with an above average population density and with an above average population.

In a sperate table I identified the 30 countries with the lowest population growth. To ensure the desired countries were targeted I removed all countries with 'None' listed for population growth, birth rate, or death rate. These were removed because the countries which reported 'None' were outlier such as the Vatican City, remote islands, and countries which did not supply the adequate data.

I then took the intersect of the two tables I had created to identify which countries, if any, had a high population with a high population density, and an extremely low population growth.

In [55]:
%%sql

WITH
avg_density AS (
SELECT AVG(population)/AVG(area) avg_pop_density
  FROM facts
 WHERE name <> 'World'
   ),
    
avg_pop AS (
SELECT AVG(population) avg_population
  FROM facts
 WHERE name <> 'World'
   ),

high_pop_popdens AS (
SELECT *,
       population/area pop_density
  FROM facts
 WHERE (pop_density > (SELECT avg_pop_density
                       FROM avg_density))
   AND (population > (SELECT avg_population
                      FROM avg_pop))
 ORDER BY pop_density DESC
 LIMIT 30
  ),

valid_pop_info AS (
SELECT *
  FROM facts
 WHERE (population_growth <> 'None')
   AND (birth_rate <> 'None')
   AND (death_rate <> 'None')
  ),

low_pop_growth AS (
SELECT *, population/area pop_density
  FROM facts
 WHERE (name IN (SELECT name
                   FROM valid_pop_info))
 ORDER BY population_growth
 LIMIT 30
  )

SELECT *
     FROM high_pop_popdens
INTERSECT
   SELECT *
     FROM low_pop_growth
    ORDER BY population_growth;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,pop_density
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46,123
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0,492
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0,335
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24,226


Four countries were identified as having a high population density, an above average population, and a low population growth rate. The four countries identified, in order of population growth, were: Poland, South Korea, Japan, and Germany. Interestingly, South Korea, Japan, and Germany, are known for being technologically advanced and highly educated societies.